In [13]:
import cv2
import numpy as np
from keras.preprocessing import image
import tensorflow as tf
import json

In [14]:
# Load the trained model
model = tf.keras.models.load_model('saved_model/my_model')

# Load the train_indices.json file
with open('train_indices.json', 'r') as f:
    train_index = json.load(f)

In [15]:
# Load the pre-trained face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create a VideoCapture object to capture video frames
video_capture = cv2.VideoCapture(0)  # Replace '0' with the video file path if using a video file

# Define a function to process and detect faces
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

In [16]:
# Loop to continuously capture and process video frames
while True:
    # Read a frame from the video capture
    ret, frame = video_capture.read()
    
    if not ret:
        break
    
    # Detect faces
    faces = detect_faces(frame)
    
    # Process each detected face
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        preprocessed_face = cv2.resize(face_img, (64, 64))
        preprocessed_face = preprocessed_face / 255.0
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)
        
        # Perform prediction
        predictions = model.predict(preprocessed_face)
        predicted_index = np.argmax(predictions)
        predicted_label = list(train_index.keys())[list(train_index.values()).index(predicted_index)]
        
        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, predicted_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Video', frame)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the OpenCV windows
video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 26ms/step
